In [4]:
import glob
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import scipy.misc
from matplotlib import pyplot as plt
%matplotlib inline

IMAGE_HEIGHT, IMAGE_WIDTH = 240, 240
SLICES_MIN = 15
SLICES_MAX = 144

survivals = pd.read_csv("val_data/survival_evaluation.csv")
survivals.head()

,Brats17ID,Age
0,Brats17_TCIA_600_1,79.389
1,Brats17_TCIA_601_1,59.468
2,Brats17_TCIA_602_1,56.351
3,Brats17_TCIA_253_1,58.882
4,Brats17_TCIA_212_1,66.567


In [83]:
def get_patient_tumor_images(paths, patient):
    # Use images from directory GT_IMAGES_PATH (only contains tumor images)
    region_values = [211, 105, 240]
    for i in range(len(paths)):
        img = Image.open(paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        for j in range(3):
            # The bottom layer contains the others and so on
            for layer_value in region_values[j:]:
                # Only set those pixels which being the given grayscale value to 1
                patient[i, j, :, :][img == layer_value] = 1
    return patient

def parse_combined_patient(tumor_paths, flairs_paths, t1_paths, t1ce_paths, t2_paths, patient):
    # Parse images from BRAIN_IMAGES_PATH (containing 7D images)
    # assert len(paths) == SLICES_MAX - SLICES_MIN, 'Expected {} slices but found {} images paths'.format(SLICES_MAX - SLICES_MIN, len(paths))
    for i in range(len(tumor_paths)):
        # Tumor images
        img = Image.open(tumor_paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        tumor1 = img.copy()
        tumor1[tumor1!=0] = 1
        patient[i, 0, :, :] = scipy.misc.imresize(tumor1, 50)
        tumor2 = img.copy()
        tumor2[tumor2==2] = 0
        tumor2[tumor2>0] = 1
        patient[i, 1, :, :] = scipy.misc.imresize(tumor2, 50)
        tumor3 = img.copy()
        tumor3[tumor3>1] = 0
        patient[i, 2, :, :] = scipy.misc.imresize(tumor3, 50)
        
        # Flair image
        img = Image.open(flairs_paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        patient[i, 3, :, :] = scipy.misc.imresize(img, 50)
        
        # T1 image
        img = Image.open(t1_paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        patient[i, 4, :, :] = scipy.misc.imresize(img, 50)
        
        # T1ce image
        img = Image.open(t1ce_paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        patient[i, 5, :, :] = scipy.misc.imresize(img, 50)
        
        # T2 image
        img = Image.open(t2_paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        patient[i, 6, :, :] = scipy.misc.imresize(img, 50)
        
#         fig, axes = plt.subplots(1, 7, figsize=(16, 4))
#         axes[0].imshow(patient[i, 0])
#         axes[1].imshow(patient[i, 1])
#         axes[2].imshow(patient[i, 2])
#         axes[3].imshow(patient[i, 3])
#         axes[4].imshow(patient[i, 4])
#         axes[5].imshow(patient[i, 5])
#         axes[6].imshow(patient[i, 6])
#         return
    
def generate_testing_data(axis='z'):
    # x has shape (155, 240) -> use z for testing
    X = np.zeros((len(survivals), SLICES_MAX - SLICES_MIN, 7, IMAGE_HEIGHT // 2, IMAGE_WIDTH // 2), dtype=np.uint8)
    for index, row in tqdm(survivals.iterrows(), total=len(survivals)):
        # For every patient, fetch his tumor pictures and sort them properly
        glob_path = "val_data/modalities/{}_*n{}*.png".format(row['Brats17ID'], axis)
        flairs = glob.glob("val_data/modalities/{}_flair.*n{}*.png".format(row['Brats17ID'], axis))
        flairs.sort(key=lambda x : int(x.split('.')[-2]))
        t1 = glob.glob("val_data/modalities/{}_t1.*n{}*.png".format(row['Brats17ID'], axis))
        t1.sort(key=lambda x : int(x.split('.')[-2]))
        t1ce = glob.glob("val_data/modalities/{}_t1ce.*n{}*.png".format(row['Brats17ID'], axis))
        t1ce.sort(key=lambda x : int(x.split('.')[-2]))
        t2 = glob.glob("val_data/modalities/{}_t2.*n{}*.png".format(row['Brats17ID'], axis))
        t2.sort(key=lambda x : int(x.split('.')[-2]))
        tumor_images = glob.glob("val_data/tumor/{0}/{0}_{1}*n{2}*.png".format(row['Brats17ID'], 'contest', axis))
        tumor_images.sort(key=lambda x : int(x.split('.')[-2]))
        # Convert every tumor image to greyscale and add it to X
        parse_combined_patient(tumor_images[SLICES_MIN:SLICES_MAX],
                               flairs[SLICES_MIN:SLICES_MAX],
                               t1[SLICES_MIN:SLICES_MAX],
                               t1ce[SLICES_MIN:SLICES_MAX],
                               t2[SLICES_MIN:SLICES_MAX],
                               X[index])
    print('Resulting dataset has the shape: {}'.format(X.shape))
    np.save('data/val_tumors_small_n{}'.format(axis), X)
    return X

In [85]:
X = generate_testing_data()


  6%|▌         | 2/33 [00:02<00:41,  1.34s/it]

100%|██████████| 33/33 [00:44<00:00,  1.36s/it]


Resulting dataset has the shape: (33, 129, 7, 120, 120)
